In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import os

In [2]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
IMG_PATH="../facial_expressions/"

In [4]:
emotions=["anger", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

In [5]:
df=pd.read_csv(IMG_PATH+"icml_face_data.csv")

In [6]:
df=df.replace(to_replace="PrivateTest",value="Testing")
df=df.replace(to_replace="PublicTest",value="Training")

In [7]:
df

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...,...
35882,6,Testing,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
35883,3,Testing,178 174 172 173 181 188 191 194 196 199 200 20...
35884,0,Testing,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
35885,3,Testing,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...


In [8]:
face_dim=64

In [9]:
import matplotlib.pyplot as plt

In [10]:
def ret_keypoints(image):
    gray=image
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim))
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    param=[0,0,face_dim,face_dim]
    shape=predictor(gray,dlib.rectangle(*param))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(np.float32(shape.part(i).x))
        ylist.append(np.float32(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
#     plt.imshow(gray,cmap='gray')
#     plt.scatter(xlist,ylist, marker='.')
#     plt.show()
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim+1)/2

In [11]:
data={"Training":[],"Testing":[]}

In [12]:
for i in range(len(df)):
    print("\r",i+1,end=" ")
    row=df.iloc[i]
    img=np.array(list(map(np.uint8,row[2].split()))).reshape(48,48)
    data[row[1]].append([row[0],ret_keypoints(img)])

 35887            28980     34459  

In [13]:
import pickle

In [14]:
# with open("data.pkl","wb") as f:
#     pickle.dump(data, f)

In [ ]:
# with open('data.pkl', 'rb') as f:
#     data = pickle.load(f)

In [143]:
# s=np.arange(len(XXD))
# for i in range(5):
#     np.random.shuffle(s)
#     XXD=XXD[s]
#     YYD=YYD[s]

In [82]:
from tensorflow.keras.models import Sequential,model_from_json
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [72]:
model=Sequential()
model.add(Dense(1024,activation='relu', input_shape=(68*2,)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(len(emotions),activation='softmax'))

In [73]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1024)              140288    
_________________________________________________________________
batch_normalization_16 (Batc (None, 1024)              4096      
_________________________________________________________________
dropout_16 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 768)               787200    
_________________________________________________________________
batch_normalization_17 (Batc (None, 768)               3072      
_________________________________________________________________
dropout_17 (Dropout)         (None, 768)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)              

In [74]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
XDB=np.vstack(np.asarray(data["Training"])[:,1])
YDY=np.asarray(data["Training"])[:,0]
YDB=np.zeros((len(YDY),len(emotions)))
for i in range(len(YDB)):
    YDB[i][YDY[i]]=1
XTB=np.vstack(np.asarray(data["Testing"])[:,1])
YTY=np.asarray(data["Testing"])[:,0]
YTB=np.zeros((len(YTY),len(emotions)))
for i in range(len(YTB)):
    YTB[i][YTY[i]]=1

In [25]:
with open('extra_data.pkl', 'rb') as f:
    exD,eyD = pickle.load(f)

In [26]:
XDB=np.concatenate((XDB,exD))
YDB=np.concatenate((YDB,eyD))

In [75]:
s=np.arange(len(XDB))
for i in range(5):
    np.random.shuffle(s)
    XDB=XDB[s]
    YDB=YDB[s]

In [76]:
model.fit(XDB,YDB,batch_size=64,epochs=100,validation_data=(XTB,YTB))

Train on 32955 samples, validate on 3589 samples
Epoch 1/100
32955/32955 [==============================] - 10s 307us/sample - loss: 1.6506 - accuracy: 0.3841 - val_loss: 1.7176 - val_accuracy: 0.3126
Epoch 2/100
32955/32955 [==============================] - 8s 255us/sample - loss: 1.4670 - accuracy: 0.4359 - val_loss: 1.7464 - val_accuracy: 0.3110
Epoch 3/100
32955/32955 [==============================] - 8s 245us/sample - loss: 1.4218 - accuracy: 0.4525 - val_loss: 1.6110 - val_accuracy: 0.3344
Epoch 4/100
32955/32955 [==============================] - 10s 291us/sample - loss: 1.3990 - accuracy: 0.4623 - val_loss: 1.4686 - val_accuracy: 0.4118
Epoch 5/100
32955/32955 [==============================] - 8s 242us/sample - loss: 1.3825 - accuracy: 0.4707 - val_loss: 1.5474 - val_accuracy: 0.3826
Epoch 6/100
32955/32955 [==============================] - 8s 228us/sample - loss: 1.3754 - accuracy: 0.4696 - val_loss: 1.6248 - val_accuracy: 0.3605
Epoch 7/100
32955/32955 [==================

32955/32955 [==============================] - 7s 220us/sample - loss: 1.1642 - accuracy: 0.5491 - val_loss: 1.5191 - val_accuracy: 0.4074
Epoch 55/100
32955/32955 [==============================] - 7s 219us/sample - loss: 1.1587 - accuracy: 0.5520 - val_loss: 1.4774 - val_accuracy: 0.4344
Epoch 56/100
32955/32955 [==============================] - 7s 219us/sample - loss: 1.1594 - accuracy: 0.5526 - val_loss: 1.5052 - val_accuracy: 0.4071
Epoch 57/100
32955/32955 [==============================] - 7s 219us/sample - loss: 1.1542 - accuracy: 0.5495 - val_loss: 1.3894 - val_accuracy: 0.4444
Epoch 58/100
32955/32955 [==============================] - 7s 219us/sample - loss: 1.1495 - accuracy: 0.5568 - val_loss: 1.4304 - val_accuracy: 0.4363
Epoch 59/100
32955/32955 [==============================] - 7s 219us/sample - loss: 1.1506 - accuracy: 0.5547 - val_loss: 1.4014 - val_accuracy: 0.4645
Epoch 60/100
32955/32955 [==============================] - 7s 220us/sample - loss: 1.1481 - accuracy

In [313]:
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# model.save_weights("model.h5")

In [99]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")

In [100]:
model=loaded_model

In [101]:
(model.predict(XTB).argmax(axis=1)==YTB.argmax(axis=1)).sum()/len(XTB)

0.4608526051825021

In [86]:
model.predict(XTB).argmax(axis=1)

array([4, 4, 4, ..., 4, 4, 4])

In [87]:
YTB.argmax(axis=1)

array([0, 5, 6, ..., 0, 3, 2])

In [88]:
from importlib import reload

In [89]:
import udp_streamer

In [90]:
reload(udp_streamer)

<module 'udp_streamer' from '/home/archer/machine_learning/expression_recognition/udp_streamer.py'>

In [91]:
from udp_streamer import *

In [93]:
handler = udp_handler()
handler.make_listener('0.0.0.0',5555)

In [94]:
from mtcnn import MTCNN
detector = MTCNN()

In [95]:
def face_rectangles(rgb):
    faces = detector.detect_faces(rgb)
    all_recs = []
    for x in faces:
        rec = x['box']
        all_recs.append(dlib.rectangle(left=rec[0], top=rec[1], right=rec[0]+rec[2], bottom=rec[1]+rec[3]))
    return all_recs

In [96]:
def get_img():
    global handler,img
    buffer = handler.get_data()
    if buffer is not None:
        try:
            npimg = np.frombuffer(buffer, dtype=np.uint8)
            img = cv2.imdecode(npimg, 1)
        except Exception as e:
            print(e)
    return img

In [97]:
# cam = cv2.VideoCapture(0)

while True:
#     ret, img = cam.read()
    img=get_img()
    try:
        grimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    except:
        continue
    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     faces = face_cascade.detectMultiScale(gimg, 1.3, 5)
    faces = face_rectangles(gimg)
#     for (x,y,w,h) in faces:
    for facrec in faces:
        x=facrec.left()
        y=facrec.top()
        w=facrec.right()-x
        h=facrec.bottom()-y
        landmarks=[]
        svx,svy=x,y
        gray=grimg[y:y+int(1.1*h),max(0,x-int(0.15*w)):x+int(1.15*w)]
        try:
            gray=cv2.resize(gray, (face_dim, face_dim))
        except:
            continue
        shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
        xlist=[]
        ylist=[]
        for i in range(68):
            xp=shape.part(i).x
            yp=shape.part(i).y
            cv2.circle(gray, (xp, yp), 1, (255, 255, 255), -1)
            xlist.append(float(xp))
            ylist.append(float(yp))
        cv2.imshow('gray',gray)
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        res=[]
        for (x,y) in zip(xcentral,ycentral):
            res.append(x)
            res.append(y)
        landmarks.append((np.asarray(res)/face_dim+1)/2)
        if len(landmarks)>0:
            y_out=model.predict(np.asarray(landmarks))
            res=np.argmax(y_out,axis=1)
            for r in res:
                cv2.putText(img,emotions[r],(svx,svy),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2,cv2.LINE_AA)
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xff == 27:
        break

# cam.release()
cv2.destroyAllWindows()

In [98]:
# cam.release()
cv2.destroyAllWindows()